<a href="https://colab.research.google.com/github/ThalesCavalcanti/MachineLearningGroundwork/blob/main/Parte_5_7_To%CC%81picos_complementares_Detecc%CC%A7a%CC%83o_de_outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 5.2 MB/s eta 0:00:00


---

Reconecte ao Drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
DRIVE_DIRECTORY = "curso_ml"
DRIVE_DIRECTORY = os.path.join("/content/drive/MyDrive/cursoML", DRIVE_DIRECTORY)

---

In [5]:
import numpy as np
import pandas as pd
import pickle
import plotly.express as px

from pyod.models.knn import KNN

## Detecção de outliers

Mais uma vez, recupere o arquivo `cov_types.csv` com os dados da base de cobertura vegetal.

In [6]:
base = pd.read_csv(os.path.join(DRIVE_DIRECTORY, "cov_types.csv"))
base

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2767.0,66.0,17.0,210.0,18.0,1190.0,234.0,204.0,96.0,2251.0,2,30,Lodgepole Pine
1,2724.0,160.0,19.0,60.0,4.0,1350.0,236.0,240.0,127.0,2514.0,2,16,Lodgepole Pine
2,2360.0,65.0,7.0,127.0,21.0,1377.0,227.0,226.0,134.0,339.0,3,5,Ponderosa Pine
3,2995.0,45.0,4.0,285.0,30.0,5125.0,221.0,231.0,146.0,5706.0,0,11,Lodgepole Pine
4,2400.0,106.0,27.0,150.0,63.0,342.0,253.0,196.0,51.0,811.0,2,3,Ponderosa Pine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2917.0,90.0,9.0,247.0,25.0,4095.0,235.0,225.0,121.0,3901.0,0,28,Lodgepole Pine
9996,3015.0,38.0,8.0,361.0,74.0,4846.0,220.0,223.0,138.0,1611.0,0,28,Lodgepole Pine
9997,3052.0,79.0,19.0,90.0,11.0,1003.0,241.0,203.0,85.0,1490.0,2,22,Spruce/Fir
9998,2958.0,58.0,6.0,319.0,19.0,2468.0,225.0,227.0,137.0,2280.0,0,28,Lodgepole Pine


Crie boxplots para as 3 primeiras variáveis.

In [7]:
fig = px.box(base["Elevation"])
fig.show()

In [8]:
fig = px.box(base["Aspect"])
fig.show()

In [9]:
fig = px.box(base["Slope"])
fig.show()

Crie scatterplots com a combinação das 3 primeiras variáveis.

In [11]:
fig = px.scatter(x=base["Elevation"], y=base["Aspect"])
fig.show()

In [12]:
fig = px.scatter(x=base["Elevation"], y=base["Slope"])
fig.show()

In [13]:
fig = px.scatter(x=base["Aspect"], y=base["Slope"])
fig.show()

Crie um detector de outliers do tipo `KNN`, e passe os dados para o método `fit`. Passe somente as variáveis preditoras numéricas. Você pode selecionar variáveis em sequência com a construção `dataframe.loc[:, "var_1":"var_n"]`.

In [14]:
detector = KNN()
detector.fit(base.loc[:, "Elevation":"Horizontal_Distance_To_Fire_Points"])

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

Recupere e exiba os rótulos, e depois sua frequência.

In [15]:
labels = detector.labels_
labels

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
np.unique(labels, return_counts=True)

(array([0, 1]), array([9000, 1000]))

Recupere e exiba também os scores de decisão.

In [17]:
confidence = detector.decision_scores_
confidence

array([128.35108102, 162.95091285, 218.40329668, ..., 135.57285864,
       193.45542122, 160.15617378])

Recupere os índices dos outliers. Utilize a função `np.where` para isso. Lembre-se que esta função retorna uma tupla com um único elemento, então recupere este elemento através do índice `0`.

In [18]:
indices_outliers = np.where(labels == 1)[0]
indices_outliers

array([  11,   13,   44,   48,   50,   71,   83,   93,   95,   99,  100,
        104,  108,  115,  118,  119,  136,  142,  155,  156,  162,  190,
        209,  210,  211,  220,  238,  247,  255,  258,  260,  265,  275,
        337,  340,  346,  361,  393,  400,  429,  430,  432,  441,  445,
        463,  464,  467,  482,  483,  498,  515,  546,  554,  556,  561,
        575,  602,  620,  621,  623,  628,  640,  645,  648,  666,  671,
        681,  685,  719,  733,  753,  756,  766,  772,  781,  787,  791,
        798,  807,  810,  818,  821,  840,  845,  861,  865,  871,  888,
        891,  913,  917,  931,  932,  934,  951,  959,  978,  991,  997,
       1000, 1002, 1007, 1009, 1035, 1043, 1061, 1065, 1073, 1079, 1080,
       1084, 1089, 1101, 1105, 1109, 1111, 1114, 1124, 1137, 1144, 1148,
       1161, 1162, 1177, 1182, 1191, 1194, 1196, 1199, 1200, 1202, 1209,
       1214, 1228, 1247, 1249, 1251, 1261, 1281, 1285, 1286, 1307, 1358,
       1363, 1369, 1370, 1373, 1389, 1394, 1402, 14

Filtre e exiba as instâncias da base que representam os outliers.

In [19]:
base_outliers = base.loc[indices_outliers]
base_outliers

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
11,2957.0,43.0,11.0,664.0,194.0,1615.0,222.0,215.0,126.0,1958.0,2,32,Lodgepole Pine
13,2533.0,13.0,9.0,0.0,0.0,484.0,211.0,221.0,146.0,2095.0,0,19,Lodgepole Pine
44,3215.0,281.0,20.0,210.0,61.0,4606.0,162.0,238.0,213.0,3424.0,0,28,Spruce/Fir
48,3201.0,199.0,17.0,750.0,117.0,404.0,212.0,252.0,170.0,2469.0,2,22,Lodgepole Pine
50,2832.0,335.0,28.0,285.0,126.0,2964.0,147.0,191.0,176.0,1775.0,0,28,Lodgepole Pine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9947,3159.0,320.0,15.0,987.0,-43.0,1385.0,181.0,225.0,183.0,1148.0,2,30,Spruce/Fir
9977,3155.0,333.0,7.0,30.0,3.0,3464.0,203.0,230.0,166.0,4842.0,0,18,Spruce/Fir
9982,2988.0,324.0,6.0,342.0,23.0,4956.0,205.0,233.0,168.0,5544.0,0,11,Lodgepole Pine
9988,3210.0,43.0,8.0,0.0,0.0,124.0,222.0,223.0,136.0,830.0,1,15,Lodgepole Pine
